In [45]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu


In [4]:
top = pd.read_csv("top_docking_scores.csv")
bottom = pd.read_csv("bottom_docking_scores.csv")

top_scores = top["Best_Affinity "].values
bottom_scores = bottom["Best_Affinity "].values

print("Top 50 count:", len(top_scores))
print("Bottom 50 count:", len(bottom_scores))


Top 50 count: 50
Bottom 50 count: 50


In [5]:
u_stat, p_value = mannwhitneyu(
    top_scores,
    bottom_scores,
    alternative="less"
)

print("Mann–Whitney U statistic:", u_stat)
print("p-value:", p_value)


Mann–Whitney U statistic: 714.5
p-value: 0.00011289150987066019


In [6]:
def cliffs_delta(x, y):
    x = np.array(x)
    y = np.array(y)
    gt = np.sum(x[:, None] < y)
    lt = np.sum(x[:, None] > y)
    return (gt - lt) / (len(x) * len(y))

delta = cliffs_delta(top_scores, bottom_scores)
print("Cliff's Delta:", delta)


Cliff's Delta: 0.4284


In [7]:
abs_delta = abs(delta)

if abs_delta < 0.147:
    size = "negligible"
elif abs_delta < 0.33:
    size = "small"
elif abs_delta < 0.474:
    size = "medium"
else:
    size = "large"

print("Effect size:", size)


Effect size: medium


In [56]:
import pandas as pd
from scipy.stats import spearmanr

top_pred = pd.read_csv("top_50_predictions.csv")
bot_pred = pd.read_csv("bottom_50_predictions.csv")
top_dock = pd.read_csv("top_docking_scores.csv")
bot_dock = pd.read_csv("bottom_docking_scores.csv")

top_combined = pd.DataFrame({
    "predicted_activity": top_pred["predicted_activity"].values,
    "Best_Affinity": top_dock["Best_Affinity"].values
})

bot_combined = pd.DataFrame({
    "predicted_activity": bot_pred["predicted_activity"].values,
    "Best_Affinity": bot_dock["Best_Affinity"].values
})

df = pd.concat([top_combined, bot_combined], ignore_index=True)

df["binding_strength"] = -df["Best_Affinity"]
df = df.dropna()

rho, pval = spearmanr(df["predicted_activity"], df["binding_strength"])

print(f"Merged rows (Position-based): {len(df)}") 
print(f"Spearman rho: {rho:.3f}")
print(f"p-value: {pval:.6f}")


Merged rows (Position-based): 100
Spearman rho: 0.347
p-value: 0.000400

Final Data Spread Check:
    predicted_activity  Best_Affinity  binding_strength
0             0.999861         -5.487             5.487
99            0.700020         -5.877             5.877


In [7]:
import numpy as np

def paired_permutation_test(a, b, n_permutations=10000, seed=42):
    rng = np.random.default_rng(seed)
    observed = np.mean(a - b)

    count = 0
    for _ in range(n_permutations):
        signs = rng.choice([-1, 1], size=len(a))
        permuted = np.mean(signs * (a - b))
        if permuted >= observed:
            count += 1

    p_value = (count + 1) / (n_permutations + 1)
    return observed, p_value



gnn = np.array([0.869, 0.875, 0.81, 0.881, 0.869])
rf  = np.array([0.634, 0.717, 0.697, 0.927, 0.598])

effect, p = paired_permutation_test(gnn, rf)

print("Mean performance difference:", effect)
print("Permutation p-value:", p)


Mean performance difference: 0.14620000000000002
Permutation p-value: 0.06259374062593741


In [8]:
import numpy as np

def paired_permutation_test(a, b, n_permutations=10000, seed=42):
    rng = np.random.default_rng(seed)
    observed = np.mean(a - b)

    count = 0
    for _ in range(n_permutations):
        signs = rng.choice([-1, 1], size=len(a))
        permuted = np.mean(signs * (a - b))
        if permuted >= observed:
            count += 1

    p_value = (count + 1) / (n_permutations + 1)
    return observed, p_value



gnn = np.array([0.869, 0.875, 0.81, 0.881, 0.869])
gcn = np.array([0.634, 0.444, 0.528, 0.771, 0.628])

effect, p = paired_permutation_test(gnn, gcn)

print("Mean performance difference:", effect)
print("Permutation p-value:", p)


Mean performance difference: 0.2598
Permutation p-value: 0.030496950304969503


In [9]:
import numpy as np
from scipy.stats import friedmanchisquare

gnn = np.array([0.869, 0.875, 0.881, 0.881, 0.869])
gcn = np.array([0.634, 0.444, 0.528, 0.771, 0.628])
gin = np.array([0.586, 0.355, 0.611, 0.844, 0.557])
rf  = np.array([0.634, 0.717, 0.697, 0.927, 0.598])
log = np.array([0.667, 0.658, 0.562, 0.896, 0.680])
xgb = np.array([0.506, 0.599, 0.577, 0.809, 0.671])

stat, p = friedmanchisquare(gnn, gcn, gin, rf, log, xgb)

print("Friedman chi-square:", stat)
print("p-value:", p)



Friedman chi-square: 14.367816091954014
p-value: 0.01343431687973739
